In [37]:
!pip3 install pgmpy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [38]:
import numpy as np 
import pandas as pd
from pgmpy.estimators import BIC
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import HillClimbSearch


data = pd.DataFrame(np.random.randint(0, 3, size=(2500, 8)), columns=list ('ABCDEFGH'))
data['A'] += data['B'] + data['C']
data['H'] = data['G'] - data['A'] + data['F']
data['G'] += data['D'] + data['E']

# Usando redes bayesianas
# Aprenda la red de dependencias entre variables usando BIC y Hill Climbing.
bic = BIC(data)
hc = HillClimbSearch(data)
best_model = hc.estimate(scoring_method=BIC(data))
print(best_model.edges())

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'A': 'N', 'B': 'N', 'C': 'N', 'D': 'N', 'E': 'N', 'F': 'N', 'G': 'N', 'H': 'N'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'A': 'N', 'B': 'N', 'C': 'N', 'D': 'N', 'E': 'N', 'F': 'N', 'G': 'N', 'H': 'N'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'A': 'N', 'B': 'N', 'C': 'N', 'D': 'N', 'E': 'N', 'F': 'N', 'G': 'N', 'H': 'N'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'A': 'N', 'B': 'N', 'C': 'N', 'D': 'N', 'E': 'N', 'F': 'N', 'G': 'N', 'H': 'N'}
  0%|          | 7/1000000 [00:00<1:40:35, 165.69it/s]

[('A', 'H'), ('A', 'C'), ('A', 'B'), ('C', 'B'), ('D', 'G'), ('E', 'G'), ('F', 'H')]


In [39]:
# Use las dependencias aprendidas para crear una red Bayesiana.
model = DiscreteBayesianNetwork([('A', 'H'), ('A', 'C'), ('A', 'B'), ('B', 'C'), ('D', 'G'), ('E', 'G'), ('F', 'H')])

print(bic.score(model))

-23211.189599012832


In [40]:
# Ajuste los parámetros de la red usando priors de Dirichlet con cuentas uniformes. Obtenga los cpds de la red.
from pgmpy.estimators import BayesianEstimator

est = BayesianEstimator(model, data)

for node in model.nodes():
    print(est.estimate_cpd(node, prior_type='BDeu', equivalent_sample_size=10))

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'A': 'N', 'B': 'N', 'C': 'N', 'D': 'N', 'E': 'N', 'F': 'N', 'G': 'N', 'H': 'N'}


+------+-----------+
| A(0) | 0.0483779 |
+------+-----------+
| A(1) | 0.115709  |
+------+-----------+
| A(2) | 0.216107  |
+------+-----------+
| A(3) | 0.249573  |
+------+-----------+
| A(4) | 0.219693  |
+------+-----------+
| A(5) | 0.111326  |
+------+-----------+
| A(6) | 0.0392146 |
+------+-----------+
+-------+-----------------------+-----+-----------------------+
| A     | A(0)                  | ... | A(6)                  |
+-------+-----------------------+-----+-----------------------+
| F     | F(0)                  | ... | F(2)                  |
+-------+-----------------------+-----+-----------------------+
| H(-6) | 0.0012556504269211452 | ... | 0.0014686444411807905 |
+-------+-----------------------+-----+-----------------------+
| H(-5) | 0.0012556504269211452 | ... | 0.0014686444411807905 |
+-------+-----------------------+-----+-----------------------+
| H(-4) | 0.0012556504269211452 | ... | 0.27287413717139086   |
+-------+-----------------------+-----+------

In [53]:
# Muestre las independencias locales de todas las variables de la red. Explique.
print(model.local_independencies('A'))
print(model.local_independencies('B'))
print(model.local_independencies('C'))
print(model.local_independencies('D'))
print(model.local_independencies('E'))
print(model.local_independencies('F'))
print(model.local_independencies('G'))
print(model.local_independencies('H'))

(A ⟂ D, F, E, G)
(B ⟂ D, H, F, E, G | A)
(C ⟂ D, H, F, E, G | A, B)
(D ⟂ B, E, H, A, F, C)
(E ⟂ B, D, H, A, F, C)
(F ⟂ B, D, A, E, C, G)
(G ⟂ B, H, A, F, C | D, E)
(H ⟂ B, D, E, C, G | A, F)


Explique: 

In [56]:
# Obtenga la cpd de H. ¿Cuál es la cardinalidad de H?
print(est.estimate_cpd('H', prior_type='BDeu', equivalent_sample_size=10))

+-------+-----------------------+-----+-----------------------+
| A     | A(0)                  | ... | A(6)                  |
+-------+-----------------------+-----+-----------------------+
| F     | F(0)                  | ... | F(2)                  |
+-------+-----------------------+-----+-----------------------+
| H(-6) | 0.0012556504269211452 | ... | 0.0014686444411807905 |
+-------+-----------------------+-----+-----------------------+
| H(-5) | 0.0012556504269211452 | ... | 0.0014686444411807905 |
+-------+-----------------------+-----+-----------------------+
| H(-4) | 0.0012556504269211452 | ... | 0.27287413717139086   |
+-------+-----------------------+-----+-----------------------+
| H(-3) | 0.0012556504269211452 | ... | 0.3067998237626671    |
+-------+-----------------------+-----+-----------------------+
| H(-2) | 0.0012556504269211452 | ... | 0.4085768835364959    |
+-------+-----------------------+-----+-----------------------+
| H(-1) | 0.0012556504269211452 | ... | 

In [60]:
# Dispone de la siguiente evidencia: 'B': O, 'C: 0, 'D': 1, 'E': O, 'F: 1. Determine cual es el resultado más probable para H.

from pgmpy.inference import VariableElimination


# First, estimate CPDs for each node and add them to the model
est = BayesianEstimator(model, data)

# Create CPDs for each node and add them to the model
cpds = []
for node in model.nodes():
    cpd = est.estimate_cpd(node, prior_type='BDeu', equivalent_sample_size=10)
    cpds.append(cpd)

# Add the CPDs to the model
model.add_cpds(*cpds)

# Check if the model is valid
model.check_model()

infer = VariableElimination(model)

print(infer.query(['H'], evidence={'B': 0, 'C': 0, 'D': 1, 'E': 0, 'F': 1}))



INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'A': 'N', 'B': 'N', 'C': 'N', 'D': 'N', 'E': 'N', 'F': 'N', 'G': 'N', 'H': 'N'}


+-------+----------+
| H     |   phi(H) |
+=======+==========+
| H(-6) |   0.0006 |
+-------+----------+
| H(-5) |   0.0008 |
+-------+----------+
| H(-4) |   0.0010 |
+-------+----------+
| H(-3) |   0.0012 |
+-------+----------+
| H(-2) |   0.0012 |
+-------+----------+
| H(-1) |   0.1038 |
+-------+----------+
| H(0)  |   0.2218 |
+-------+----------+
| H(1)  |   0.3143 |
+-------+----------+
| H(2)  |   0.2750 |
+-------+----------+
| H(3)  |   0.0797 |
+-------+----------+
| H(4)  |   0.0006 |
+-------+----------+
